In [0]:
from torchvision.datasets import utils
import torch.utils.data as data_utils
import torch
import os
import numpy as np
from torch import nn
from torch.nn.modules import upsampling
from torch.functional import F
from torch.optim import Adam


from torchvision.utils import save_image

# To do this assignment, I did look at this code for the general framework of how to implement VAE and I did copy some code from there:
https://github.com/pytorch/examples/blob/master/vae/main.py

In [2]:
# If a GPU is available, use it
# Pytorch uses an elegant way to keep the code device agnostic
if torch.cuda.is_available():
    device = torch.device("cuda")
    use_cuda = True
else:
    device = torch.device("cpu")
    use_cuda = False
    
print(device)

torch.manual_seed(1) # I may need to fix other seeds

cuda


In [0]:
def get_data_loader(dataset_location, batch_size):
    URL = "http://www.cs.toronto.edu/~larocheh/public/datasets/binarized_mnist/"
    # start processing
    def lines_to_np_array(lines):
        return np.array([[int(i) for i in line.split()] for line in lines])
    splitdata = []
    for splitname in ["train", "valid", "test"]:
        filename = "binarized_mnist_%s.amat" % splitname
        filepath = os.path.join(dataset_location, filename)
        utils.download_url(URL + filename, dataset_location)
        with open(filepath) as f:
            lines = f.readlines()
        x = lines_to_np_array(lines).astype('float32')
        x = x.reshape(x.shape[0], 1, 28, 28)
        # pytorch data loader
        dataset = data_utils.TensorDataset(torch.from_numpy(x))
        print(splitname, len(dataset))
        dataset_loader = data_utils.DataLoader(x, batch_size=batch_size, shuffle=splitname == "train")
        splitdata.append(dataset_loader)
    return splitdata

In [4]:
batch_size = 64
train, valid, test = get_data_loader("binarized_mnist", 64)

  0%|          | 0/78400000 [00:00<?, ?it/s]

 73%|███████▎  | 57548800/78400000 [00:03<00:02, 9817325.41it/s] 

KeyboardInterrupt: ignored

In [0]:
print(f"Your version of Pytorch is {torch.__version__}")

In [0]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
for x in train:
    plt.imshow(x[0, 0])
    break

In [0]:
for x in train:
    print(x.shape)
    plt.imshow(x[0, 0])
    break

In [0]:
print(train)

In [0]:
class Q2_VAE(nn.Module):
    def __init__(self):
        super(Q2_VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400,20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar
        


In [0]:
model = Q2_VAE()
model = model.to(device)

#optimizer = Adam(model.parameters(), lr=1e-3)
optimizer = Adam(model.parameters(), lr=3 * 10**(-4))


print(model)
print("\n\n# Parameters: ", sum([param.nelement() for param in model.parameters()]))

In [0]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    #BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    #BCE = - F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    BCE = - torch.sum(F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='none'),dim=1).mean()
    
    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    
    #KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    #KLD = 0.5 * torch.sum(-1 - logvar + mu.pow(2) + logvar.exp())
    KLD = 0.5 * torch.sum(-1 - logvar + mu.pow(2) + logvar.exp(), dim = 1).mean()

    #return BCE + KLD
    return - (BCE - KLD)




def train_VAE(epoch,loader):
    model.train()
    train_loss = 0
    #for batch_idx, (data, _) in enumerate(loader):
    for batch_idx, data in enumerate(loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(loader.dataset),
                100. * batch_idx / len(loader),
                loss.item() / len(data)))
        
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss * batch_size / len(loader.dataset)))


def test_VAE(epoch, loader):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        #for i, (data, _) in enumerate(loader):
        for i, data in enumerate(loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                        recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                            str(epoch) + '.png', nrow=n)

    test_loss /= (len(loader.dataset)/ batch_size)
    print('====> Test set loss: {:.4f}'.format(test_loss))


    

In [0]:
len(train.dataset)

In [0]:
nb_epochs = 50
for epoch in range(1, nb_epochs + 1):
    train_VAE(epoch, train)
    test_VAE(epoch, valid)
    with torch.no_grad():
        sample = torch.randn(64, 20).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                    str(epoch) + '.png')

In [0]:
 a = torch.tensor([[0.5,0.5,0.5],[0.5,0.5,0.5]], dtype=torch.float64)
 b = torch.tensor([[1,1,1],[0,0,0]], dtype=torch.float64)
 print(torch.sum(F.binary_cross_entropy(a, b, reduction='none')+0,dim = 1))
 
 print(F.binary_cross_entropy(a, b) * 6)

In [0]:
0.7 * 6